In [7]:
import cvxpy as cp
import numpy as np
import csv
from datetime import datetime

In [8]:
def DCOPF(Y,PGl,PGu,PD,thetaL,thetaU,CQ,CL,PF,slack,gens_to_buses,eff_in,eff_out,eff_store,max_charge,b_cost,b_lifetime,max_b_size):
    num_buses = len(Y)
    num_generators = PGl.shape[1]
    B = np.imag(Y)
    t = len(PD)

    PG = cp.Variable((t,num_generators)); # optimization variable: PG_i
    theta = cp.Variable((t,num_buses)); # optimization variable: theta_i
    S = cp.Variable((t,num_buses));
    qR = cp.Variable((t,num_buses));
    qD = cp.Variable((t,num_buses));
    Smaxs = cp.Variable(num_buses);

    objective = cp.Minimize(cp.sum(PG**2 * CQ.T + PG * CL.T) + (t/b_lifetime) * b_cost * cp.sum(Smaxs)) # objective function

    constraints = [
        # Power generation constraints
        PG - PGu <= 0,
        -PG + PGl <= 0,

        #Bus phase angle constraints
        theta.T - thetaU <= 0,
        -theta.T + thetaL <= 0,

        #Power balance constraint
        qR + theta * B + PD == PG * gens_to_buses + qD*eff_out,

        #Slack angle constraint   
        theta[:,slack] == 0,
        
        # Storage dynamics
        S[1:,:] == eff_store*S[:-1,:] + eff_in*qR[:-1,:] - qD[:-1,:],
        
        # Storage and charge limits
        S >= 0,
        qR <= max_charge,
        qD <= max_charge,
        qR >= 0,
        qD >= 0,
        S[0,:] == S[t-1,:],
        
        # Constraints on battery capacities
        Smaxs >= 0,
        Smaxs <= max_b_size
    ]
    
    for i in range(num_buses):
        constraints.append(S[:,i] <= Smaxs[i])
    
    # Line capacity constraints
    count = 0
    for i in range(num_buses):          
        for j in range(i+1, num_buses):
            constraints.append(B[i,j] * (theta[:,i] - theta[:,j]) - PF[:,count] <= 0)
            constraints.append(B[i,j] * (theta[:,j] - theta[:,i]) - PF[:,count] <= 0)
            count = count + 1

    # SOLVE IT
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    
    pf_opt = np.zeros((t,num_buses,num_buses))
    for i in range(num_buses):
        for j in range(i+1, num_buses):
            pf_opt[:,i,j] = B[i,j] * (theta[:,i].value - theta[:,j].value)
    
    cost = objective.value
    p_opt = PG.value
    theta_opt = theta.value
    lmp_opt = constraints[4].dual_value.reshape((t,num_buses))
    qD_opt = qD.value
    qR_opt = qR.value
    S_opt = S.value
    Smaxs_opt = Smaxs.value
    
    return pf_opt, cost, p_opt, theta_opt, lmp_opt, qD_opt, qR_opt, S_opt, Smaxs_opt

In [9]:
def load_load_profiles(filename, loads, start_date, num_hours):
    with open(filename) as load_profiles_file:
        reader = csv.reader(load_profiles_file)
        next(reader)
        col_to_bus = [7,4,2,3,0,5,6,1] # VERY FILE SPECIFIC, MAKE SURE IT'S CORRECT BEFORE LOADING FILE
        t = 0

        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break

            if t >= num_hours:
                break

            day = datetime.strptime(next_line[0], '%m/%d/%y')
            if day >= start_date:
                for col in range(2,len(next_line)):
                    loads[t,col_to_bus[col-2]] = float(next_line[col])
                t += 1
            

In [10]:
def load_cost_curves(filename, cost_curves_dict):
    with open(filename) as curves_file:
        reader = csv.reader(curves_file)
        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break

            fuel = next_line[0]
            q = float(next_line[1])
            l = float(next_line[2])
            c = float(next_line[3])
            cost_curves_dict[fuel] = (q,l,c)

In [11]:
def load_wind_curve(regidxs):
    wind = {'02/16/2018 00:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 459.91305666110424, 'North': 442.5268610173908, 'NorthCentral': 230.89302130433526, 'Northwest': 447.3043324198788, 'South': 531.7787131275077, 'SouthCentral': 0.0, 'West': 638.8510764633824}, '02/16/2018 00:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 450.4351783511342, 'North': 433.40727705066195, 'NorthCentral': 226.13478292243096, 'Northwest': 438.08629442596737, 'South': 520.8198289256859, 'SouthCentral': 0.0, 'West': 625.6856473171188}, '02/16/2018 00:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 438.252663452706, 'North': 421.68530047439884, 'NorthCentral': 220.01871895937654, 'Northwest': 426.2377686776495, 'South': 506.73368372603653, 'SouthCentral': 0.0, 'West': 608.7632907017314}, '02/16/2018 00:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 425.75280758799784, 'North': 409.65797944303455, 'NorthCentral': 213.74333833108125, 'Northwest': 414.0806020090198, 'South': 492.2806101075767, 'SouthCentral': 0.0, 'West': 591.4001255139898}, '02/16/2018 01:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 418.05325280398375, 'North': 402.2494926422089, 'NorthCentral': 209.87788280414838, 'Northwest': 406.5921339981377, 'South': 483.3779288825146, 'SouthCentral': 0.0, 'West': 580.7049108624068}, '02/16/2018 01:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 417.0122850676035, 'North': 401.247876840855, 'NorthCentral': 209.35527927669452, 'Northwest': 405.57970486256886, 'South': 482.17430033741886, 'SouthCentral': 0.0, 'West': 579.2589346081596}, '02/16/2018 01:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 414.3004859996331, 'North': 398.63859251661563, 'NorthCentral': 207.99385787126715, 'Northwest': 402.94225099126214, 'South': 479.03875765660234, 'SouthCentral': 0.0, 'West': 575.4920579591197}, '02/16/2018 01:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 397.8422966328386, 'North': 382.80257574554986, 'NorthCentral': 199.7312503782639, 'Northwest': 386.93527032191645, 'South': 460.0088244222172, 'SouthCentral': 0.0, 'West': 552.6304934931155}, '02/16/2018 02:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 388.5032486017616, 'North': 373.81657382578163, 'NorthCentral': 195.0427098274558, 'Northwest': 377.8522565120774, 'South': 449.2104640106717, 'SouthCentral': 0.0, 'West': 539.6579092157504}, '02/16/2018 02:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 383.09474906037445, 'North': 368.6125329963262, 'NorthCentral': 192.3274470582273, 'Northwest': 372.59203342922626, 'South': 442.95683653823215, 'SouthCentral': 0.0, 'West': 532.1451289108135}, '02/16/2018 02:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 381.1241547347846, 'North': 366.71643348662144, 'NorthCentral': 191.33813729410838, 'Northwest': 370.6754638374047, 'South': 440.6783186762556, 'SouthCentral': 0.0, 'West': 529.4078369641242}, '02/16/2018 02:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 377.11327654437065, 'North': 362.85717941713995, 'NorthCentral': 189.32453109168486, 'Northwest': 366.77454568473183, 'South': 436.0407037798792, 'SouthCentral': 0.0, 'West': 523.8364494759938}, '02/16/2018 03:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 389.43858618513474, 'North': 374.7165526343126, 'NorthCentral': 195.51228319015263, 'Northwest': 378.7619513930085, 'South': 450.2919567172214, 'SouthCentral': 0.0, 'West': 540.95715787447}, '02/16/2018 03:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 400.9305353585411, 'North': 385.77406909535597, 'NorthCentral': 201.28165813372783, 'Northwest': 389.9388435876598, 'South': 463.5796289288497, 'SouthCentral': 0.0, 'West': 556.9202708884651}, '02/16/2018 03:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 410.6037816847359, 'North': 395.0816355377092, 'NorthCentral': 206.13797833974877, 'Northwest': 399.3468934953621, 'South': 474.76440920113436, 'SouthCentral': 0.0, 'West': 570.357079735009}, '02/16/2018 03:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 428.42425675520803, 'North': 412.2284392227929, 'NorthCentral': 215.08450262408073, 'Northwest': 416.67881219032614, 'South': 495.36949784352856, 'SouthCentral': 0.0, 'West': 595.1109533573642}, '02/16/2018 04:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 451.86277138769157, 'North': 434.7809024699192, 'NorthCentral': 226.8514863615478, 'Northwest': 439.4747493544281, 'South': 522.4704965396077, 'SouthCentral': 0.0, 'West': 627.6686728802055}, '02/16/2018 04:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 477.3746389541887, 'North': 459.32833922862903, 'NorthCentral': 239.6593684084477, 'Northwest': 464.2871975450982, 'South': 551.9688286863033, 'SouthCentral': 0.0, 'West': 663.1064231710342}, '02/16/2018 04:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 498.5540408940923, 'North': 479.7070915231033, 'NorthCentral': 250.29219570590504, 'Northwest': 484.88595661177476, 'South': 576.4577074978289, 'SouthCentral': 0.0, 'West': 692.5260787606954}, '02/16/2018 04:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 516.1543864329856, 'North': 496.64208728228454, 'NorthCentral': 259.1282069880762, 'Northwest': 502.00378072572465, 'South': 596.808269339279, 'SouthCentral': 0.0, 'West': 716.9741772236508}, '02/16/2018 05:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 529.7496493608418, 'North': 509.7234054597842, 'NorthCentral': 265.95352165869804, 'Northwest': 515.2263233779515, 'South': 612.5279174764547, 'SouthCentral': 0.0, 'West': 735.8589386594707}, '02/16/2018 05:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 539.299305256788, 'North': 518.912053588431, 'NorthCentral': 270.74779498992143, 'Northwest': 524.5141711428897, 'South': 623.5697951740015, 'SouthCentral': 0.0, 'West': 749.1240718419705}, '02/16/2018 05:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 538.4041592700652, 'North': 518.0507470046767, 'NorthCentral': 270.2983992652526, 'Northwest': 523.643565987828, 'South': 622.5347743717296, 'SouthCentral': 0.0, 'West': 747.8806520935481}, '02/16/2018 05:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 552.3480434104887, 'North': 531.4675073894524, 'NorthCentral': 277.29873441832126, 'Northwest': 537.2051722445693, 'South': 638.657519000955, 'SouthCentral': 0.0, 'West': 767.2496725294135}, '02/16/2018 06:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 578.4429988835358, 'North': 556.5759894528031, 'NorthCentral': 290.3993477249221, 'Northwest': 562.5847227233869, 'South': 668.8300519168924, 'SouthCentral': 0.0, 'West': 803.4973722908585}, '02/16/2018 06:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 593.8323283496884, 'North': 571.3835526719469, 'NorthCentral': 298.12534881322375, 'Northwest': 577.5521467691873, 'South': 686.6241060340301, 'SouthCentral': 0.0, 'West': 824.8742163554222}, '02/16/2018 06:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 601.7685620772793, 'North': 579.0197711895677, 'NorthCentral': 302.10962574687704, 'Northwest': 585.2708050634744, 'South': 695.8004494702792, 'SouthCentral': 0.0, 'West': 835.8981944454209}, '02/16/2018 06:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 606.6443608135584, 'North': 583.7112490210137, 'NorthCentral': 304.55745340731437, 'Northwest': 590.0129315744686, 'South': 701.4381367241938, 'SouthCentral': 0.0, 'West': 842.6710164520507}, '02/16/2018 07:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 615.8210565722372, 'North': 592.541036107472, 'NorthCentral': 309.16448723386947, 'Northwest': 598.9380440728096, 'South': 712.0487758235505, 'SouthCentral': 0.0, 'West': 855.4180821830611}, '02/16/2018 07:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 612.6007966621985, 'North': 589.4425123995451, 'NorthCentral': 307.54780005953256, 'Northwest': 595.806069043147, 'South': 708.3253206050169, 'SouthCentral': 0.0, 'West': 850.9449182225604}, '02/16/2018 07:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 616.3510269189038, 'North': 593.0509718671681, 'NorthCentral': 309.4305515535735, 'Northwest': 599.4534850429768, 'South': 712.6615589243219, 'SouthCentral': 0.0, 'West': 856.1542476855575}, '02/16/2018 07:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 624.2510828270443, 'North': 600.6523802034994, 'NorthCentral': 313.3966659108092, 'Northwest': 607.1369573490794, 'South': 721.7960714231963, 'SouthCentral': 0.0, 'West': 867.1279722796725}, '02/16/2018 08:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 627.7147416769574, 'North': 603.9851015870034, 'NorthCentral': 315.1355481735348, 'Northwest': 610.5056584267097, 'South': 725.8009589106082, 'SouthCentral': 0.0, 'West': 871.9392342186858}, '02/16/2018 08:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 630.8551095215101, 'North': 607.0067534069775, 'NorthCentral': 316.71212663577677, 'Northwest': 613.5599316679576, 'South': 729.4320381916513, 'SouthCentral': 0.0, 'West': 876.3014225691284}, '02/16/2018 08:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 623.7642174126181, 'North': 600.1839198706963, 'NorthCentral': 313.1522418291761, 'Northwest': 606.6634395699598, 'South': 721.2331287978456, 'SouthCentral': 0.0, 'West': 866.4516825123042}, '02/16/2018 08:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 618.8859921225314, 'North': 595.4901072169963, 'NorthCentral': 310.70319595079104, 'Northwest': 601.9189530302269, 'South': 715.5926358187129, 'SouthCentral': 0.0, 'West': 859.67548985444}, '02/16/2018 09:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 624.7913041774483, 'North': 601.1721794459571, 'NorthCentral': 313.66787660587704, 'Northwest': 607.662368287085, 'South': 722.4207073415748, 'SouthCentral': 0.0, 'West': 867.8783771360588}, '02/16/2018 09:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 630.9394461455905, 'North': 607.0879018348869, 'NorthCentral': 316.75446667735304, 'Northwest': 613.6419561653847, 'South': 729.5295531910047, 'SouthCentral': 0.0, 'West': 876.4185719787802}, '02/16/2018 09:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 630.8767340353702, 'North': 607.0275604445271, 'NorthCentral': 316.72298292538864, 'Northwest': 613.5809633360001, 'South': 729.4570417352253, 'SouthCentral': 0.0, 'West': 876.3314605159912}, '02/16/2018 09:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 625.9807692919397, 'North': 602.3166789459864, 'NorthCentral': 314.2650311985622, 'Northwest': 608.8192236780516, 'South': 723.7960373495689, 'SouthCentral': 0.0, 'West': 869.5306265292924}, '02/16/2018 10:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 610.6677733020595, 'North': 587.5825635517219, 'NorthCentral': 306.57735228161, 'Northwest': 593.9260403917834, 'South': 706.090244518326, 'SouthCentral': 0.0, 'West': 848.2598149480003}, '02/16/2018 10:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 597.671761878481, 'North': 575.0778432405086, 'NorthCentral': 300.0528835825037, 'Northwest': 581.2863204930975, 'South': 691.0634864593275, 'SouthCentral': 0.0, 'West': 830.2074553390811}, '02/16/2018 10:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 582.8281910733558, 'North': 560.79540724626, 'NorthCentral': 292.6008731198704, 'Northwest': 566.8496928880537, 'South': 673.9004708270423, 'SouthCentral': 0.0, 'West': 809.5887078387198}, '02/16/2018 10:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 568.7033427777845, 'North': 547.2045237345708, 'NorthCentral': 285.50968740292956, 'Northwest': 553.1120836902392, 'South': 657.5684847245582, 'SouthCentral': 0.0, 'West': 789.96831566282}, '02/16/2018 11:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 546.1297373240576, 'North': 525.4842733120138, 'NorthCentral': 274.17691941677595, 'Northwest': 531.1573438289845, 'South': 631.4675452426933, 'SouthCentral': 0.0, 'West': 758.6120148687775}, '02/16/2018 11:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 522.1237144779635, 'North': 502.3857554905862, 'NorthCentral': 262.1250259900583, 'Northwest': 507.8094569453534, 'South': 603.7103599410108, 'SouthCentral': 0.0, 'West': 725.265987147428}, '02/16/2018 11:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 500.55152205935275, 'North': 481.62906146329453, 'NorthCentral': 251.29500363789498, 'Northwest': 486.82867592840967, 'South': 578.7673135161277, 'SouthCentral': 0.0, 'West': 695.3007183891204}, '02/16/2018 11:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 472.70904722297166, 'North': 454.839121900047, 'NorthCentral': 237.3170722822757, 'Northwest': 459.74951511914884, 'South': 546.5741952205411, 'SouthCentral': 0.0, 'West': 656.6255932475145}, '02/16/2018 12:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 452.6401156694138, 'North': 435.52886063274616, 'NorthCentral': 227.24174136127212, 'Northwest': 440.23078239145576, 'South': 523.3693080340938, 'SouthCentral': 0.0, 'West': 628.748457904719}, '02/16/2018 12:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 423.35658155911807, 'North': 407.35233848005214, 'NorthCentral': 212.5403460273776, 'Northwest': 411.7500695993583, 'South': 489.50995166345416, 'SouthCentral': 0.0, 'West': 588.0716016640409}, '02/16/2018 12:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 398.31942089385313, 'North': 383.26166317193184, 'NorthCentral': 199.97078404787314, 'Northwest': 387.39931400574625, 'South': 460.5605037491017, 'SouthCentral': 0.0, 'West': 553.2932521239942}, '02/16/2018 12:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 373.80762804657826, 'North': 359.67649508524715, 'NorthCentral': 187.6649757518854, 'Northwest': 363.5595230340282, 'South': 432.21851721929363, 'SouthCentral': 0.0, 'West': 519.2446748554695}, '02/16/2018 13:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 351.8775408199337, 'North': 338.5754358270064, 'NorthCentral': 176.65527723628378, 'Northwest': 342.23065905691305, 'South': 406.8616516220806, 'SouthCentral': 0.0, 'West': 488.7822654309827}, '02/16/2018 13:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 337.85087167241943, 'North': 325.07901997518843, 'NorthCentral': 169.61338100959688, 'Northwest': 328.5885373814533, 'South': 390.64318606495505, 'SouthCentral': 0.0, 'West': 469.2982508888858}, '02/16/2018 13:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 329.2784913201208, 'North': 316.8307032253007, 'NorthCentral': 165.30973541692424, 'Northwest': 320.25117270959163, 'South': 380.7312922272813, 'SouthCentral': 0.0, 'West': 457.39062109538116}, '02/16/2018 13:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 317.2684556619723, 'North': 305.2746856182083, 'NorthCentral': 159.28026228299217, 'Northwest': 308.57039760525356, 'South': 366.8445777398258, 'SouthCentral': 0.0, 'West': 440.7078500858477}, '02/16/2018 14:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 308.71792131553576, 'North': 297.0473890247608, 'NorthCentral': 154.9875841769431, 'Northwest': 300.25428002113296, 'South': 356.9579435478961, 'SouthCentral': 0.0, 'West': 428.8305659069311}, '02/16/2018 14:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 295.92196435375075, 'North': 284.73516047199564, 'NorthCentral': 148.56355006746173, 'Northwest': 287.8091300007839, 'South': 342.16250030526794, 'SouthCentral': 0.0, 'West': 411.05609579563975}, '02/16/2018 14:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 285.35822104149526, 'North': 274.57076069934345, 'NorthCentral': 143.26016810358016, 'Northwest': 277.53499648422826, 'South': 329.9480746806004, 'SouthCentral': 0.0, 'West': 396.3823249844532}, '02/16/2018 14:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 278.0528614162767, 'North': 267.54156720998947, 'NorthCentral': 139.59261283166202, 'Northwest': 270.42991659376236, 'South': 321.50118524320186, 'SouthCentral': 0.0, 'West': 386.2346746994082}, '02/16/2018 15:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 269.2731071551064, 'North': 259.0937159532623, 'NorthCentral': 135.18485802168067, 'Northwest': 261.89086326243415, 'South': 311.34951341097286, 'SouthCentral': 0.0, 'West': 374.03898818954315}, '02/16/2018 15:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 254.8958014648976, 'North': 245.25991875001438, 'NorthCentral': 127.96692954378912, 'Northwest': 247.9077179035186, 'South': 294.7256211177523, 'SouthCentral': 0.0, 'West': 354.06791521432837}, '02/16/2018 15:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 243.015552842845, 'North': 233.8287818108057, 'NorthCentral': 122.00261420534969, 'Northwest': 236.35317166504288, 'South': 280.9889740876914, 'SouthCentral': 0.0, 'West': 337.56542738336526}, '02/16/2018 15:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 236.98005038476458, 'North': 228.02144079546864, 'NorthCentral': 118.97257324988536, 'Northwest': 230.48313523374512, 'South': 274.01036871054305, 'SouthCentral': 0.0, 'West': 329.1816966183931}, '02/16/2018 16:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 234.47592612027677, 'North': 225.61198049789365, 'NorthCentral': 117.71541212176558, 'Northwest': 228.04766266735413, 'South': 271.1149519364506, 'SouthCentral': 0.0, 'West': 325.7032946491598}, '02/16/2018 16:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 237.70734189287523, 'North': 228.72123834082504, 'NorthCentral': 119.33769994338647, 'Northwest': 231.19048771656406, 'South': 274.8513062239493, 'SouthCentral': 0.0, 'West': 330.19195487509995}, '02/16/2018 16:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 239.4103340913999, 'North': 230.35985194623427, 'NorthCentral': 120.19266374204376, 'Northwest': 232.84679161453894, 'South': 276.8204066586537, 'SouthCentral': 0.0, 'West': 332.55752894063016}, '02/16/2018 16:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 232.8473480198845, 'North': 224.0449678979028, 'NorthCentral': 116.89780689707476, 'Northwest': 226.46373277139034, 'South': 269.2318934889658, 'SouthCentral': 0.0, 'West': 323.44108691778206}, '02/16/2018 17:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 229.4016148440619, 'North': 220.72949454024194, 'NorthCentral': 115.16792397235382, 'Northwest': 223.11246592739622, 'South': 265.2477327275332, 'SouthCentral': 0.0, 'West': 318.65472498111296}, '02/16/2018 17:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 226.5845428523281, 'North': 218.0189169480068, 'NorthCentral': 113.75365174419541, 'Northwest': 220.3726252370943, 'South': 261.99046725777805, 'SouthCentral': 0.0, 'West': 314.7416169526968}, '02/16/2018 17:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 225.3606136843776, 'North': 216.841256246802, 'NorthCentral': 113.13919494772567, 'Northwest': 219.18225064026538, 'South': 260.57528787014104, 'SouthCentral': 0.0, 'West': 313.04149460318877}, '02/16/2018 17:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 217.93756017486345, 'North': 209.6988180812555, 'NorthCentral': 109.41255308076425, 'Northwest': 211.962703496517, 'South': 251.99231379364122, 'SouthCentral': 0.0, 'West': 302.7303593646587}, '02/16/2018 18:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 209.04234751550229, 'North': 201.13987312586178, 'NorthCentral': 104.94683397077628, 'Northwest': 203.31135711114942, 'South': 241.70714212373537, 'SouthCentral': 0.0, 'West': 290.37429314627553}, '02/16/2018 18:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 197.05786255468118, 'North': 189.60844031739893, 'NorthCentral': 98.93018821283823, 'Northwest': 191.65543222022745, 'South': 227.8499708656957, 'SouthCentral': 0.0, 'West': 273.7270138242591}, '02/16/2018 18:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 191.75801148100663, 'North': 184.50894069344926, 'NorthCentral': 96.26947091173007, 'Northwest': 186.50087895825817, 'South': 221.72196918601554, 'SouthCentral': 0.0, 'West': 266.36515376293994}, '02/16/2018 18:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 189.37431590498795, 'North': 182.21535649182945, 'NorthCentral': 95.07276934945568, 'Northwest': 184.18253347343116, 'South': 218.96579919357063, 'SouthCentral': 0.0, 'West': 263.05403557952496}, '02/16/2018 19:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 185.60446777235234, 'North': 178.58802076720616, 'NorthCentral': 93.1801689707625, 'Northwest': 180.5160374306031, 'South': 214.60687752430317, 'SouthCentral': 0.0, 'West': 257.81745552867335}, '02/16/2018 19:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 181.9938753641962, 'North': 175.1139203874644, 'NorthCentral': 91.36752073705088, 'Northwest': 177.00443104461473, 'South': 210.43209675513702, 'SouthCentral': 0.0, 'West': 252.80209270473696}, '02/16/2018 19:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 180.3414673540781, 'North': 173.52397872512887, 'NorthCentral': 90.53795203410189, 'Northwest': 175.39732454666682, 'South': 208.52148475476997, 'SouthCentral': 0.0, 'West': 250.50678357895464}, '02/16/2018 19:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 180.70117940272488, 'North': 173.87009249913962, 'NorthCentral': 90.71854051818319, 'Northwest': 175.7471749269788, 'South': 208.9374051283174, 'SouthCentral': 0.0, 'West': 251.00644851815676}, '02/16/2018 20:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 178.7077140205836, 'North': 171.95198653252461, 'NorthCentral': 89.71774865484768, 'Northwest': 173.80836129895457, 'South': 206.63244239628423, 'SouthCentral': 0.0, 'West': 248.23738709050556}, '02/16/2018 20:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 179.98911386883984, 'North': 173.18494533711632, 'NorthCentral': 90.36105781551979, 'Northwest': 175.05463099142284, 'South': 208.11407278802497, 'SouthCentral': 0.0, 'West': 250.01734019377594}, '02/16/2018 20:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 190.20925891472586, 'North': 183.01873596513715, 'NorthCentral': 95.4919409979731, 'Northwest': 184.99458614332235, 'South': 219.93120974850603, 'SouthCentral': 0.0, 'West': 264.213829224935}, '02/16/2018 20:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 200.71948313860045, 'North': 193.13163984341818, 'NorthCentral': 100.76845444000092, 'Northwest': 195.21666782148543, 'South': 232.08375343367484, 'SouthCentral': 0.0, 'West': 278.81325831711985}, '02/16/2018 21:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 207.14473469173694, 'North': 199.3139961820245, 'NorthCentral': 103.99416356536118, 'Northwest': 201.46576820030614, 'South': 239.51300979628596, 'SouthCentral': 0.0, 'West': 287.7383775582854}, '02/16/2018 21:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 208.2426010728208, 'North': 200.3703596759524, 'NorthCentral': 104.54533227441503, 'Northwest': 202.53353607852668, 'South': 240.78242792404512, 'SouthCentral': 0.0, 'West': 289.2633899692403}, '02/16/2018 21:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 206.87175155235659, 'North': 199.05133268501996, 'NorthCentral': 103.85711613671398, 'Northwest': 201.20026901702815, 'South': 239.19737052390025, 'SouthCentral': 0.0, 'West': 287.3591850784815}, '02/16/2018 21:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 208.71752010506674, 'North': 200.8273252383197, 'NorthCentral': 104.78375883926682, 'Northwest': 202.99543499093093, 'South': 241.33155743482916, 'SouthCentral': 0.0, 'West': 289.92308538468666}, '02/16/2018 22:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 201.0037966599015, 'North': 193.40520539739387, 'NorthCentral': 100.91119013097575, 'Northwest': 195.49318675914458, 'South': 232.4124935646462, 'SouthCentral': 0.0, 'West': 279.208189481838}, '02/16/2018 22:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 196.24090623414435, 'North': 188.82236757847716, 'NorthCentral': 98.52004653412217, 'Northwest': 190.86087312632637, 'South': 226.90535758602405, 'SouthCentral': 0.0, 'West': 272.5922049354062}, '02/16/2018 22:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 193.45567331820303, 'North': 186.1424254423659, 'NorthCentral': 97.1217586758328, 'Northwest': 188.1519986291689, 'South': 223.68490634127704, 'SouthCentral': 0.0, 'West': 268.7233235875529}, '02/16/2018 22:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 191.98766451811673, 'North': 184.72991210568367, 'NorthCentral': 96.3847650587913, 'Northwest': 186.72423595359658, 'South': 221.98750762805682, 'SouthCentral': 0.0, 'West': 266.6841577308551}, '02/16/2018 23:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 200.20250445098995, 'North': 192.63420461619924, 'NorthCentral': 100.50891240394974, 'Northwest': 194.7138623381714, 'South': 231.48599205849726, 'SouthCentral': 0.0, 'West': 278.0951391284925}, '02/16/2018 23:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 194.56890324940895, 'North': 187.21357169471105, 'NorthCentral': 97.68064044380998, 'Northwest': 189.2347088588435, 'South': 224.97208871554898, 'SouthCentral': 0.0, 'West': 270.2696770332786}, '02/16/2018 23:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 203.36283302525715, 'North': 195.67506258597973, 'NorthCentral': 102.09551187586834, 'Northwest': 197.78754907671015, 'South': 235.14015111735233, 'SouthCentral': 0.0, 'West': 282.4850543144323}, '02/16/2018 23:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 214.15266459988607, 'North': 206.0570037561024, 'NorthCentral': 107.51239834070891, 'Northwest': 208.28157254377754, 'South': 247.61599338039713, 'SouthCentral': 0.0, 'West': 297.4728773746278}}
    from datetime import datetime

    wind_curve = np.zeros((24, 9))
    for time in sorted(wind.keys()):
        h = datetime.strptime(time, '%m/%d/%Y %H:%M:%S').hour
        for reg in wind[time].keys():
            wind_curve[h, regidxs[reg]] += wind[time][reg]

    return wind_curve

In [12]:
def load_generators(filename, generator_dict, cost_curves_dict, wind_curve):
    with open(filename) as generators_file:
        reader = csv.reader(generators_file)
        next(reader)
        count = 0

        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break

            fuel = next_line[0]
            weather_zone = next_line[1]
            jan_mar = float(next_line[2])
            apr_sep = float(next_line[3])
            oct_dec = float(next_line[4])
            gen_id = "{} {}".format(weather_zone, fuel)

            generator_dict[gen_id] = {
                            'idx'   : count,
                            'bus'   : weather_zone,
                            'p_nom' : apr_sep,
                            'cq'    : cost_curves_dict[fuel][0] * SBASE * SBASE,
                            'cl'    : SBASE * cost_curves_dict[fuel][1],
                            'c0'    : 1 * cost_curves_dict[fuel][2],
            }
            count += 1

In [88]:
SBASE = 1e3 #MW
NUM_HOURS = 24
START_DATE = datetime.strptime('05/01/17', '%m/%d/%y')
NUM_BUSES = 9

wind_curve_rand = np.random.rand((NUM_HOURS))
solar_curve = np.random.rand((NUM_HOURS))

# Useful data structures to transfer between bus indexes and names
regidxs = {"North" : 0,
         "West" : 1,
         "FarWest" : 2,
         "NorthCentral" : 3,
         "East" : 4,
         "SouthCentral" : 5,
         "South" : 6,
         "Coast" : 7,
         "Northwest" : 8}

regnames = ['North', 'West', 'FarWest', 'NorthCentral', 'East', 'SouthCentral', 'South', 'Coast', 'Northwest']

# distances of transmission lines, in km
distances = {}
distances[('FarWest', 'South')] = 579
distances[('FarWest', 'West')] = 224
distances[('West', 'North')] = 195
distances[('North', 'NorthCentral')] = 198
distances[('East', 'NorthCentral')] = 146
distances[('East', 'Coast')] = 290
distances[('West', 'SouthCentral')] = 340
distances[('SouthCentral', 'Coast')] = 243
distances[('NorthCentral', 'SouthCentral')] = 241 #note: needs fixing
distances[('South', 'SouthCentral')] = 193
distances[('South', 'Coast')] = 391
distances[('Northwest', 'North')] = 200 # made up

# Construct Y-bus
impedance_per_km = .005j

Y = np.zeros((NUM_BUSES, NUM_BUSES),dtype=complex)
for b1 in range(NUM_BUSES):
    for b2 in range(b1+1, NUM_BUSES):
        if (regnames[b1],regnames[b2]) in distances.keys() or (regnames[b2],regnames[b1]) in distances.keys():
            #Y[b1,b2] = 1/(distances[(regnames[b1],regnames[b2])]*impedance_per_km)
            Y[b1,b2] = 1/impedance_per_km
        #elif (regnames[b2],regnames[b1]) in distances.keys():
         #   Y[b1,b2] = 1/(distances[(regnames[b2],regnames[b1])]*impedance_per_km)
        else:
            Y[b1,b2] = 0
        Y[b2,b1] = Y[b1,b2]
    Y[b1,b1] = -1*np.sum(Y[b1,:])
    
# set voltage angle constraints
thetal = -np.ones((NUM_BUSES,NUM_HOURS))
thetau = np.ones((NUM_BUSES,NUM_HOURS))
    
# Get cost curves
cost_curves_dict = {}
load_cost_curves('cost_quadratic_estimates.csv', cost_curves_dict)
    
# Get all the generators
wind_curve = load_wind_curve(regidxs)
generators_dict = {}
load_generators('zonal_generator_capacities_wo_biomass.csv', generators_dict, cost_curves_dict, wind_curve)
num_gens = len(generators_dict.keys())

# Gens to buses matrix
gens_to_buses = np.zeros((num_gens, NUM_BUSES))
PGl = np.zeros((NUM_HOURS,num_gens))
PGu = np.zeros((NUM_HOURS,num_gens))
CQ = np.zeros((1,num_gens))
CL = np.zeros((1,num_gens))

for gen in generators_dict.keys():
    gen_idx = generators_dict[gen]['idx']
    bus_idx = regidxs[generators_dict[gen]['bus']]
    gens_to_buses[gen_idx, bus_idx] = 1
    PGu[:,gen_idx] = generators_dict[gen]['p_nom']
    if 'WIND' in gen:
        PGu[:,gen_idx] = wind_curve[:,bus_idx]
        #PGu[:,gen_idx] = PGu[:,gen_idx] * wind_curve_rand
        #PGl[:,gen_idx] = 0 #PGu[:,gen_idx]
    elif 'SOLAR' in gen:
        PGu[:,gen_idx] = PGu[:,gen_idx] * solar_curve
        #PGl[:,gen_idx] = 0 #PGu[:,gen_idx]
    CQ[0,gen_idx] = generators_dict[gen]['cq']
    CL[0,gen_idx] = generators_dict[gen]['cl']

PGl = PGl / SBASE
PGu = PGu / SBASE

line_capacity = 1000
PF = line_capacity * np.ones((NUM_HOURS, num_gens)) / SBASE
slack= 0

PD = np.zeros((NUM_HOURS,NUM_BUSES))
load_load_profiles('load_profiles_processed.csv', PD, START_DATE, NUM_HOURS)
PD = PD/SBASE
print('Total demand over time period: ', np.sum(PD)*SBASE)

Total demand over time period:  526094.8040000001


In [91]:
EFF_IN = .9
EFF_OUT = .9
EFF_STORE = 1
battery_cost = 291 * 1000 * SBASE # $/MWh capacity, https://www.lazard.com/media/450338/lazard-levelized-cost-of-storage-version-30.pdf
MAX_CHARGE = 100/SBASE # Lazard ^
BATTERY_LIFETIME = 10*365*24 # hours

In [92]:
(pf_opt, cost, p_opt, theta_opt, 
 lmp_opt, qD_opt, qR_opt, S_opt, Smaxs_opt) = DCOPF(Y, PGl, PGu, PD, thetal, thetau, 
                                                    CQ, CL, PF, slack, gens_to_buses, 
                                                    EFF_IN, EFF_OUT, EFF_STORE, MAX_CHARGE,
                                                    battery_cost, BATTERY_LIFETIME,10000)
np.set_printoptions(precision=3, suppress=True)
#print(np.matmul(p_opt**2, CQ.T) + np.matmul(p_opt, CL.T))
#print('OPF:', pf_opt)
#print('Total cost (optimizer):', cost)
print('Total cost (real): ', np.sum(np.matmul(p_opt, gens_to_buses) * lmp_opt))
#print('Generation:', p_opt * SBASE)
#print('Angles:', theta_opt)
print('LMPs: ', lmp_opt)
#print('qD: ', qD_opt)
#print('qR: ', qR_opt)
print('S: ', S_opt * SBASE)
#print('Angles:', theta_opt)
print('Battery capacities: ', Smaxs_opt*SBASE)

Total cost (real):  7663353.824688146
LMPs:  [[ 9950.448 10987.959 16310.185 12947.114  1572.17     47.279 -1000.
    288.165  8861.244]
 [10165.788 11388.207 16571.054 19079.372 20744.213 22142.487 22800.772
  23631.838 22927.924]
 [24084.942 23112.743 22978.188 22112.389 21938.942 20539.185 10856.951
  11780.748 16364.642]
 [13462.419  3264.564  2094.559     0.        0.     9406.569 10597.721
  11395.997 16689.29 ]
 [19122.138 20744.213 22142.487 22800.772 23897.889 22927.924 24702.127
  23112.743 22978.188]
 [22112.389 21938.942 20539.185 11212.066 12243.209 16396.409 13763.015
    632.282    47.279]
 [-1000.    -1000.     3703.285  4298.86   4697.999 14873.312 19147.085
  20744.213 22142.487]
 [22800.772 24806.895 22927.924 26810.843 23112.743 22978.188 22112.389
  21938.942 20539.185]
 [ 9043.945 10195.169 16255.727 12431.808  -120.225 -2000.    -2000.
    576.33   8315.918]
 [ 9733.855 11380.417 16452.819 19036.606 20744.213 22142.487 22800.772
  23365.788 22927.924]
 [23467.757

In [62]:
print('Regions: ', regnames)
(pf_opt, cost, p_opt, theta_opt, 
     lmp_opt, qD_opt, qR_opt, S_opt, Smaxs_opt) = DCOPF(Y, PGl, PGu, PD, thetal, thetau, 
                                                        CQ, CL, PF, slack, gens_to_buses, 
                                                        EFF_IN, EFF_OUT, EFF_STORE, MAX_CHARGE,
                                                        5, BATTERY_LIFETIME,10000)
print('Baseline total cost: ', np.sum(np.matmul(p_opt, gens_to_buses) * lmp_opt))

for bcost in range(0, int(360001 * SBASE), int(30000 * SBASE)):
    (pf_opt, cost, p_opt, theta_opt, 
     lmp_opt, qD_opt, qR_opt, S_opt, Smaxs_opt) = DCOPF(Y, PGl, PGu, PD, thetal, thetau, 
                                                        CQ, CL, PF, slack, gens_to_buses, 
                                                        EFF_IN, EFF_OUT, EFF_STORE, MAX_CHARGE,
                                                        bcost, BATTERY_LIFETIME, 10000)
    print('Total cost (real): ', np.sum(np.matmul(p_opt, gens_to_buses) * lmp_opt) +
                                    np.sum(NUM_HOURS*Smaxs_opt*SBASE*bcost/BATTERY_LIFETIME))
    np.set_printoptions(precision=2, suppress=True)
    print('Cost: ', bcost/(1000*SBASE), '$/kWh\tCapacities: ', Smaxs_opt*SBASE+.001)

Regions:  ['North', 'West', 'FarWest', 'NorthCentral', 'East', 'SouthCentral', 'South', 'Coast', 'Northwest']
Baseline total cost:  14525363.85734966
Total cost (real):  14525460.389524281
Cost:  0.0 $/kWh	Capacities:  [479193.79 479193.62 479193.52 479193.79 479192.88 479193.46 479193.46
 479193.46 479193.39]
Total cost (real):  29917384.939136233
Cost:  30.0 $/kWh	Capacities:  [270.01 180.    90.   524.42   0.   101.43   0.     0.   740.26]
Total cost (real):  26139246.17275545
Cost:  60.0 $/kWh	Capacities:  [  0.     0.     0.     0.     0.     0.     0.     0.   726.14]
Total cost (real):  31294359.08608476
Cost:  90.0 $/kWh	Capacities:  [  0.     0.     0.     0.     0.     0.     0.     0.   691.91]
Total cost (real):  19161647.923466705
Cost:  120.0 $/kWh	Capacities:  [  0.     0.     0.     0.     0.     0.     0.     0.   150.41]
Total cost (real):  14209529.306249145
Cost:  150.0 $/kWh	Capacities:  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
Total cost (real):  14209784.749696275
Cost:  180